# TaPL Chapter 10 - An ML Implementation of Simple Types

Chapter 10 of [TaPL](https://www.cis.upenn.edu/~bcpierce/tapl/) is all about 
implementing the Simply Typed Lambda Calculus described in [chapter
9](09 - Simply Typed Lambda Calculus.ipynb).

My Python implementation based on the `inference` library can be found
[here](tapl/chapter_09_simply_typed_lambda_calculus.py). _(Warning: Mojibake if
viewed via jupyter or nbview. Github renders it correctly.)_

In [1]:
from tapl import SimplyTypedLambdaCalculus

## QuickCheck Terms

For this chapter, Leo wrote a [Haskell QuickCheck
generator](https://github.com/leocassarani/types-and-programming-languages/blob/master/09-simply-typed-lambda-calculus/Generator.hs)
which creates random terms with a given type.

Here is a quick test of my code against some of those terms which he shared on
Slack.

### 1

In [2]:
random_term_1 = """
    (
        (λs:(((Bool → Bool) → (Bool → Bool)) → (Bool → ((Bool → Bool) → (Bool → Bool)))). false)
        (λv:((Bool → Bool) → (Bool → Bool)). (λj:Bool. v))
    )
"""
type_1 = SimplyTypedLambdaCalculus.infer_type(random_term_1)
assert type_1 == ('Bool',)
type_1.proof

"(s : (((Bool → Bool) → (Bool → Bool)) → (Bool → ((Bool → Bool) → (Bool → Bool)))) , ∅) ⊢ false : Bool"
∅ ⊢ (λ s : (((Bool → Bool) → (Bool → Bool)) → (Bool → ((Bool → Bool) → (Bool → Bool)))) . false) : ((((Bool → Bool) → (Bool → Bool)) → (Bool → ((Bool → Bool) → (Bool → Bool)))) → Bool)
"v : ((Bool → Bool) → (Bool → Bool)) ∈ (v : ((Bool → Bool) → (Bool → Bool)) , ∅)"
"v : ((Bool → Bool) → (Bool → Bool)) ∈ (j : Bool , (v : ((Bool → Bool) → (Bool → Bool)) , ∅))"
"(j : Bool , (v : ((Bool → Bool) → (Bool → Bool)) , ∅)) ⊢ v : ((Bool → Bool) → (Bool → Bool))"
"(v : ((Bool → Bool) → (Bool → Bool)) , ∅) ⊢ (λ j : Bool . v) : (Bool → ((Bool → Bool) → (Bool → Bool)))"
∅ ⊢ (λ v : ((Bool → Bool) → (Bool → Bool)) . (λ j : Bool . v)) : (((Bool → Bool) → (Bool → Bool)) → (Bool → ((Bool → Bool) → (Bool → Bool))))
∅ ⊢ ((λ s : (((Bool → Bool) → (Bool → Bool)) → (Bool → ((Bool → Bool) → (Bool → Bool)))) . false) (λ v : ((Bool → Bool) → (Bool → Bool)) . (λ j : Bool . v))) : Bool


### 2

In [3]:
random_term_2 = """
    (if (if true then false else (if false then true else (if false then false else false))) then true else true)
"""
type_2 = SimplyTypedLambdaCalculus.infer_type(random_term_2)
assert type_2 == ('Bool',)
type_2.proof

∅ ⊢ true : Bool
∅ ⊢ false : Bool
∅ ⊢ false : Bool
∅ ⊢ true : Bool
∅ ⊢ false : Bool
∅ ⊢ false : Bool
∅ ⊢ false : Bool
∅ ⊢ (if false then false else false) : Bool
∅ ⊢ (if false then true else (if false then false else false)) : Bool
∅ ⊢ (if true then false else (if false then true else (if false then false else false))) : Bool
∅ ⊢ true : Bool


### 3

In [4]:
random_term_3 = """
    (
        if false then 
            (λt:Bool. (
                (λn:(((Bool → Bool) → (Bool → Bool)) → ((Bool → Bool) → ((Bool → Bool) → Bool))). t)
                (λt':((Bool → Bool) → (Bool → Bool)). (λy:(Bool → Bool). (λj:(Bool → Bool). t)))
            ))
        else
            (λc:Bool. c)
    )
"""
type_3 = SimplyTypedLambdaCalculus.infer_type(random_term_3)
assert type_3 == ('Bool','→','Bool')
type_3.proof

∅ ⊢ false : Bool
"t : Bool ∈ (t : Bool , ∅)"
"t : Bool ∈ (n : (((Bool → Bool) → (Bool → Bool)) → ((Bool → Bool) → ((Bool → Bool) → Bool))) , (t : Bool , ∅))"
"(n : (((Bool → Bool) → (Bool → Bool)) → ((Bool → Bool) → ((Bool → Bool) → Bool))) , (t : Bool , ∅)) ⊢ t : Bool"
"(t : Bool , ∅) ⊢ (λ n : (((Bool → Bool) → (Bool → Bool)) → ((Bool → Bool) → ((Bool → Bool) → Bool))) . t) : ((((Bool → Bool) → (Bool → Bool)) → ((Bool → Bool) → ((Bool → Bool) → Bool))) → Bool)"
"t : Bool ∈ (t : Bool , ∅)"
"t : Bool ∈ (t' : ((Bool → Bool) → (Bool → Bool)) , (t : Bool , ∅))"
"t : Bool ∈ (y : (Bool → Bool) , (t' : ((Bool → Bool) → (Bool → Bool)) , (t : Bool , ∅)))"
"t : Bool ∈ (j : (Bool → Bool) , (y : (Bool → Bool) , (t' : ((Bool → Bool) → (Bool → Bool)) , (t : Bool , ∅))))"
"(j : (Bool → Bool) , (y : (Bool → Bool) , (t' : ((Bool → Bool) → (Bool → Bool)) , (t : Bool , ∅)))) ⊢ t : Bool"
"(y : (Bool → Bool) , (t' : ((Bool → Bool) → (Bool → Bool)) , (t : Bool , ∅))) ⊢ (λ j : (Bool → Bool) . t) : ((Bool → Bool) → Bool)"


### 4

The forth term is over 18K characters long. So we'll load it from a file.

In [5]:
%%time
random_term_4 = open("data/large_lambda_term.txt").read()
type_4 = SimplyTypedLambdaCalculus.infer_type(random_term_4)
assert type_4 == ('Bool','→','Bool')
print('Characters in term:', len(random_term_4))

Characters in term: 18048
CPU times: user 32.1 s, sys: 78.7 ms, total: 32.2 s
Wall time: 32.2 s


Rendering the proof would be silly given its size. Lets looks at some stats instead.

In [6]:
f'Proof size = {type_4.proof.size}, depth = {type_4.proof.depth}, width = {type_4.proof.width}'

'Proof size = 1338, depth = 50, width = 328'

In [7]:
from collections import defaultdict
def count_rules_used_in_proof(proof):
    result = defaultdict(int)
    result[proof.rule.name] = 1
    for p in proof.premises:
        for (name, count) in count_rules_used_in_proof(p).items():
            result[name] += count
    return result

for name, count in sorted(count_rules_used_in_proof(type_4.proof).items()):
    print(f'{name:8} {count}')

M_HEAD   143
M_TAIL   295
T_ABS    374
T_APP    69
T_FALSE  82
T_IF     129
T_TRUE   103
T_VAR    143
